In [48]:
from pprint import pprint
from bs4 import BeautifulSoup as bs
import requests
import pymongo as pm

### Task 1 Парсинг вакансий с hh.ru

In [49]:
User_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 YaBrowser/22.11.0.2419 Yowser/2.5 Safari/537.36'
URL_first_page = 'https://hh.ru/search/vacancy?search_field=name&search_field=company_name&search_field=description&text=Data+Scientist&from=suggest_post&customDomain=1'

headers = {
    'User-Agent': User_agent,
}

In [50]:
def parse_hh(url_page, headers, result=[], index_page=1):
    response = requests.get(url_page, headers=headers)
    if response.status_code != 200:
        print('Парсинг завершен')
        return result
    else:
        print('Cтраница №%d, ссылка: %s'%(index_page, response.url))

    dom = bs(response.content, 'html.parser')
    vacancies = dom.find_all('div', {'class': 'vacancy-serp-item__layout'})
    for vacancy in vacancies:
        result.append(parse_vacancy_hh(vacancy))

    link_next_page = dom.find('a', {'data-qa': 'pager-next'})
    if link_next_page:
        link_next_page =  'https://hh.ru' + link_next_page['href']
    else:
        print('Парсинг завершен')
        return result

    result = parse_hh(link_next_page, headers, result, index_page+1)   
    return result
     
def parse_vacancy_hh(dom_vacancy):
    vacancy_name = dom_vacancy.find('a').text

    vacancy_salary = dom_vacancy.find('span', {'class', 'bloko-header-section-3'})
    if vacancy_salary:
        vacancy_salary = vacancy_salary.text
        min_salary, max_salary, currency_salary = clean_salary(vacancy_salary)
    else:
        min_salary, max_salary, currency_salary = None, None, None

    vacancy_link = dom_vacancy.find('a')['href']
  
    return {
          'vacancy_name': vacancy_name,
          'vacancy_salary': vacancy_salary,
          'min_salary': min_salary,
          'max_salary': max_salary,
          'currency_salary': currency_salary,
          'vacancy_link': vacancy_link,
          'vacancy_source': 'hh.ru',
      }

def clean_salary(vacancy_salary_text, min_salary=None, max_salary=None, currency_salary=None):
    list_salary = vacancy_salary_text.replace('\u202f', '').split()
    for i in range(len(list_salary) - 1):
        if list_salary[i] == 'от':
            min_salary = int(list_salary[i + 1])
        elif list_salary[i] == 'до':
            max_salary = int(list_salary[i + 1])
        elif list_salary[i] == '–':
            min_salary = int(list_salary[i - 1])
            max_salary = int(list_salary[i + 1])
        currency_salary = list_salary[-1]

    return min_salary, max_salary, currency_salary

In [51]:
result = parse_hh(URL_first_page, headers)

Cтраница №1, ссылка: https://hh.ru/search/vacancy?search_field=name&search_field=company_name&search_field=description&text=Data+Scientist&from=suggest_post&customDomain=1
Cтраница №2, ссылка: https://hh.ru/search/vacancy?search_field=name&search_field=company_name&search_field=description&text=Data+Scientist&from=suggest_post&customDomain=1&page=1&hhtmFrom=vacancy_search_list
Cтраница №3, ссылка: https://hh.ru/search/vacancy?search_field=name&search_field=company_name&search_field=description&text=Data+Scientist&from=suggest_post&customDomain=1&page=2&hhtmFrom=vacancy_search_list
Cтраница №4, ссылка: https://hh.ru/search/vacancy?search_field=name&search_field=company_name&search_field=description&text=Data+Scientist&from=suggest_post&customDomain=1&page=3&hhtmFrom=vacancy_search_list
Cтраница №5, ссылка: https://hh.ru/search/vacancy?search_field=name&search_field=company_name&search_field=description&text=Data+Scientist&from=suggest_post&customDomain=1&page=4&hhtmFrom=vacancy_search_l

In [52]:
print(f'Вакансия: {result[0]}\nКол-во вакансий: {len(result)}')

Вакансия: {'vacancy_name': 'Специалист по углубленной аналитике данных (Data scientist)', 'vacancy_salary': None, 'min_salary': None, 'max_salary': None, 'currency_salary': None, 'vacancy_link': 'https://hh.ru/vacancy/72920605?from=vacancy_search_list&query=Data+Scientist', 'vacancy_source': 'hh.ru'}
Кол-во вакансий: 143


### Task 2 Перенос данных в базу

In [53]:
client = pm.MongoClient()

db = client['vacancies_ds']
collection_of_ds_vacancies = db['Data_Scientist']

In [54]:
def check_and_save_vacancies_in_db(vacancies):
  for vacancy in vacancies:
    if not len(list(collection_of_ds_vacancies.find({'vacancy_link': vacancy['vacancy_link']}))):
      collection_of_ds_vacancies.insert_one(vacancy)

In [55]:
check_and_save_vacancies_in_db(result)
result_find = list(collection_of_ds_vacancies.find())

In [56]:
len(result_find)

143